In [5]:
import serial
import time
import numpy as np

#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui

#matplotlib -> normal plotting
import matplotlib.pyplot as plt
%matplotlib inline


In [10]:
# Change the configuration file name
configFileName = 'mmw_pplcount_demo_default.cfg'

global CLIport
global Dataport

CLIport = {}
Dataport = {}

CLIport = serial.Serial('COM4', 115200)
if not(CLIport.is_open):
    CLIport.open()
Dataport = serial.Serial('COM3', 921600)
if not(Dataport.is_open):
    Dataport.open()

In [11]:
# Read the configuration file and send it to the board
config = [line.rstrip('\r\n') for line in open(configFileName)]
for i in config:
    CLIport.write((i+'\n').encode())
    print(i)
    time.sleep(0.01)

#close control port
CLIport.close()

dfeDataOutputMode 1
channelCfg 15 3 0
adcCfg 2 1
adcbufCfg 0 1 1 1 
profileCfg 0 77 30 7 62 0 0 60 1 128 2500 0 0 30
chirpCfg 0 0 0 0 0 0 0 1
chirpCfg 1 1 0 0 0 0 0 2
frameCfg 0 1 128 0 50 1 0
lowPower 0 1
guiMonitor 1 1 0 0
cfarCfg 6 4 4 4 4 16 16 4 4 50 62 0
doaCfg 600 1875 30 1
SceneryParam -6 6 0.05 6
GatingParam 4 3 2 0
StateParam 10 5 10 100 5
AllocationParam 450 0.01 25 1 2
VariationParam 0.289 0.289 1.0
PointCloudEn 1
trackingCfg 1 2 250 20 200 50 90
sensorStart


In [26]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long

[2 1 4 3 6 5 8 7]


In [32]:
#read and parse data

while Dataport.is_open:
    
    while (not(lostSync) and Dataport.is_open):
        #check for a valid frame header
        if not(gotHeader):
            
            #in_waiting = amount of bytes in the buffer
            recieveHeader = Dataport.read(Dataport.in_waiting)
            headerVector = np.frombuffer(recieveHeader, dtype = 'uint8')
            
            #magic byte check
            if not(np.array_equal(headerVector[0:8],magicBytes)):
                print('NO SYNC PATTERN')
                lostSync = True
                break
             
            #length entire header is the same as the desired length
            if (len(headerVector) == )
            
            

SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS
SUCCESS


KeyboardInterrupt: 

In [19]:
byteBuffer = np.zeros(2**15,dtype = 'uint8')

(32768,)

In [2]:
a = 12
type(a)

int

True


In [17]:
a = [1,2,3]
b = [1,2]

print(a==b)

False
